In [2]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd

from data_utils import load_tabular_data


In [2]:
#load_tabular_data('adult', {'test':100, 'train': 5000}, 'seller_divide_train.csv', 'seller_divide_test.csv')
import os
def load_data(file_path, dict_no, train_file_name, test_file_name):
    df = pd.read_csv(file_path, header=None)

    df.columns = ['AT', 'V', 'AP', 'RH', 'PE']
    
    df.AT = df.AT.astype(float)
    df.V = df.V.astype(float)
    df.AP = df.AP.astype(float)
    df.RH = df.RH.astype(float)
    df.PE = df.PE.astype(float)

    df = df.rename(columns={'PE': 'Y'})

    df = df.reset_index()
    df = df.drop(columns=['index'])

    train_idx = range(9000)
    train = df.loc[train_idx]

    test_idx = range(9000, len(df))
    test = df.loc[test_idx]

    train_idx_final = np.random.permutation(len(train))[:dict_no['train']]

    temp_idx = np.random.permutation(len(test))
    test_idx_final = temp_idx[:dict_no['test']] + 9000
    train = train.loc[train_idx_final]
    test = test.loc[test_idx_final]

    # Saves data
    if not os.path.exists('data_files'):
        os.makedirs('data_files')

    train.to_csv('./data_files/' + str(train_file_name), index=False)
    test.to_csv('./data_files/' + str(test_file_name), index=False)

file_path = "./data_files/Folds5x2_pp.csv"
load_data(file_path, {'test':100, 'train': 5000}, 'seller_divide_train.csv', 'seller_divide_test.csv')

In [3]:
from dynashap.dynamic import mc_shap
from data_utils import preprocess_data
from sklearn import svm, linear_model
import pandas as pd
from sklearn import metrics
x_train, y_train, x_test, y_test, \
columns_name = \
    preprocess_data('seller_divide_train.csv', 'seller_divide_test.csv')
#model = svm.SVC(decision_function_shape='ovo')
model = linear_model.LinearRegression()
sv = mc_shap(x_train, y_train, x_test, y_test, model, 5000, 14)
new_idx = np.argsort(sv)
x_new = x_train[new_idx]
y_new = y_train[new_idx]
x_new_pd = pd.DataFrame(x_new)
y_new_pd = pd.DataFrame(y_new)
x_new_pd.to_csv('./data_files/' + 'x_new.csv', index=False)
y_new_pd.to_csv('./data_files/' + 'y_new.csv', index=False)

In [40]:
import random
seller_data = pd.read_csv('./data_files/' + 'seller_in2.csv')
seller_data = seller_data.values
magnification = 112
seller_aug = np.zeros((np.size(seller_data, 0) * magnification, 5))
idx = 0
for k in range(np.size(seller_data, 0)):
    for i in range(magnification):
        for j in range(np.size(seller_data, 1)):
            seller_aug[idx][j] = seller_data[k][j] + random.gauss(0, 0.1)
        idx += 1
seller_aug_pd = pd.DataFrame(seller_aug)
seller_aug_pd.to_csv('./data_files/' + 'seller_aug.csv', index=False)